In [1]:
print("Stack fitting Jupyter module.")

Stack fitting Jupyter module.


Test to include latex math: $\mathrm{F_\lambda\ [erg\, s^{-1}\, cm^{-2}\, \AA]} 
= \frac{L \times L_\odot}{4 \pi d_l^2}$

Note: \AA (outside math mode) will not work or $\AA$ (inside math mode) will not work. Check out: https://github.com/ipython/ipython/issues/5533

Therefore, there are these workaround symbols: $\mathring A$ $\unicode[serif]{xC5}$ $\unicode{x212B}$ $\mathrm{\mathring A}$. I like $\mathrm{\mathring A}$ best so I will use that for the rest of this document.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import interpolate

import emcee
import corner

import time

# Check that it imported the correct matplotlibrc
import matplotlib
print("Using matplotlibrc:", matplotlib.matplotlib_fname())

# To get figures to show up as a pop up
%matplotlib widget

# To get figures to show up within the notebook
# %matplotlib inline

import os
import sys

home = os.getenv('HOME')
figs_dir = home + "/Documents/pears_figs_data/"
stacking_analysis_dir = home + "/Documents/GitHub/stacking-analysis-pears/"
stacking_figures_dir = home + "/Documents/stacking_figures/"
massive_galaxies_dir = home + "/Documents/GitHub/massive-galaxies/"
pears_spectra_dir = home + "/Documents/pears_figs_data/data_spectra_only/"


Using matplotlibrc: /Users/baj/miniconda3/envs/astroconda37/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc


In [3]:
# Define redshift range 
z_low = 0.16
z_high = 0.96

In [4]:
# Load in saved stack
stack = np.genfromtxt(stacking_analysis_dir + 'massive_stack_pears_' + str(z_low) + 'z' + str(z_high) + '.txt', \
                      dtype=None, names=['lam', 'flam', 'flam_err'], encoding='ascii')

In [5]:
# Load models
models_llam = np.load(figs_dir + 'model_comp_spec_llam_withlines_chabrier.npy', mmap_mode='r')
models_grid = np.load(figs_dir + 'model_lam_grid_withlines_chabrier.npy', mmap_mode='r')

total_models = len(models_llam)
print("Total models:", total_models)

Total models: 37761


### Some notes on the choice of the lsf_sigma below. (This will be a hand-wavy argument)

1. It has to be in Angstroms. This is because we are convolving/filtering the spectrum which is spaced in the units of wavelength, i.e., angstroms.
2. For an object at $z\sim0.6$ (approx median of our redshift distribution) 
3. PEARS data is taken by ACS/G800L which has a spectral resolution of $R=\frac{\delta\lambda}{\lambda}\sim100$ at 8000$\mathrm{\mathring A}$. See: https://hst-docs.stsci.edu/acsihb/chapter-6-polarimetry-coronagraphy-prism-and-grism-spectroscopy/6-3-grism-and-prism-spectroscopy. This resolution is only achieved for point sources. For extended objects this resolution is even worse. 
4. So, say at the approximate middle of our stack's wavelength coverage (say 3600$\mathrm{\mathring A}$ to 6200$\mathrm{\mathring A}$), i.e., at 4900$\mathrm{\mathring A}$, *if* we could get this best spectral resolution then the spectral coverage per element or pixel would be: $\frac{4900}{R}=49\mathrm{\mathring A}$.
5. However, we do not get this best spectral resolution since the galaxies we are considering are not point sources. Therefore, I'll assume for the sake of this order-of-magnitude calculation that we get $R\sim80$ on average for the galaxies considered below.
6. Now, at 4900A and assuming an $R\sim80$ we get a coverage of $\frac{4900}{80}=61.25\mathrm{\mathring A}$ per spectral element.
7. Finally, assuming that galaxies have a 1$\sigma$ width along the dispersion direction of 2 pixels, I get the LSF sigma should be $61.25\times2 = 122.5\mathrm{\mathring A}$. This will be my LSF sigma below.

In [17]:
# Loop over all models and modify them
# ------------ NO COVARIANCE MATRIX FOR THE MOMENT ------------- # 
# First do the convolution with the LSF

# LSF defined as a Gaussian for now
lsf_sigma = 122.5  # this is in Angstroms

models_lsfconv = np.zeros(models_llam.shape)

# Seems like it takes the same amount of time for the explicit
# for loop and the vectorized computation at least for the two
# machines this has been tested on. Probably due to lack of enough 
# RAM. Vectorized should be faster on a machine with more memory.
# I'm sticking with the vectorized version for now.
# 
# ---- Use the code block below to check
"""
t1 = time.time()

for i in range(total_models):
    models_lsfconv[i] = scipy.ndimage.gaussian_filter1d(input=models_llam[i], sigma=lsf_sigma)

t2 = time.time()
print("Time taken for for loop:", "{:.2f}".format(t2 - t1), "seconds.")
    
# Without for loop over all models
models_lsfconv_nofor = scipy.ndimage.gaussian_filter1d(input=models_llam, sigma=lsf_sigma, axis=1)

t3 = time.time()
print("Time taken for vectorized computation:", "{:.2f}".format(t3 - t2), "seconds.")

print("Arrays equal:", np.array_equal(models_lsfconv_nofor, models_lsfconv))

t4 = time.time()
print("Time taken for array comparison:", "{:.2f}".format(t4 - t3), "seconds.")
"""

models_lsfconv = scipy.ndimage.gaussian_filter1d(input=models_llam, sigma=lsf_sigma, axis=1)

print("LSF convolution done.")


LSF convolution done.


In [18]:
print(models_lsfconv.shape)

(37761, 13228)


In [19]:
# Now do the resampling
# Define array to save modified models
resampling_grid = stack['lam']

models_mod = np.zeros((total_models, len(resampling_grid)))

### Zeroth element
lam_step = resampling_grid[1] - resampling_grid[0]
idx = np.where((models_grid >= resampling_grid[0] - lam_step) & \
               (models_grid < resampling_grid[0] + lam_step))[0]
models_mod[:, 0] = np.mean(models_lsfconv[:, idx], axis=1)

### all elements in between
for j in range(1, len(resampling_grid) - 1):
    # sys.stdout.write('\r' + str(j))
    # The above line will print 'j' at each iteration but not on a new line
    # Useful for seeing how fast the code is going on a slower machine
    # \r stands for carriage return... effectively flushes whatever is already printed
    # print(j, end='\r')  # Accomplishes the same thing as above without needing the sys package
    idx = np.where((models_grid >= resampling_grid[j-1]) & \
                   (models_grid < resampling_grid[j+1]))[0]
    models_mod[:, j] = np.mean(models_lsfconv[:, idx], axis=1)

### Last element
lam_step = resampling_grid[-1] - resampling_grid[-2]
idx = np.where((models_grid >= resampling_grid[-1] - lam_step) & \
               (models_grid < resampling_grid[-1] + lam_step))[0]
models_mod[:, -1] = np.mean(models_lsfconv[:, idx], axis=1)

print("Resampling done.")

Resampling done.


In [20]:
print(models_mod.shape)

(37761, 184)


In [21]:
# For all resampled models fit a polynomial and divide continuum. Rebin to a large delta-lambda first.
models_divcont = np.zeros(models_mod.shape)

# Rebin data 
rebin_step = 250.0
rebin_start = int(resampling_grid[0] / rebin_step) * rebin_step
rebin_end = int(resampling_grid[-1] / rebin_step) * rebin_step
rebin_grid = np.arange(rebin_start, rebin_end + rebin_step, rebin_step)

for k in range(total_models):
    
    model_rebinned = interpolate.griddata(points=resampling_grid, values=models_mod[k], \
    xi=rebin_grid, method='cubic')    
    
    np_fit = np.polyfit(rebin_grid, model_rebinned, deg=10)
    np_polynomial = np.poly1d(np_fit)
    
    models_divcont[k] = models_mod[k] / np_polynomial(resampling_grid)
    
    """
    # Plot to check
    print("Working on model index:", k)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(resampling_grid, models_mod[k], label='Resampled model')
    ax.plot(rebin_grid, model_rebinned, label='Rebinned model')
    ax.plot(resampling_grid, np_polynomial(resampling_grid), label='Polynomial fit')
    
    ax.legend()
    
    plt.show()
    
    if k > kinit + 10: break
    """

In [22]:
# Compute chi2 for each model
chi2 = (models_divcont - stack['flam'])**2 / stack['flam_err']**2
chi2 = np.nansum(chi2, axis=1)

In [23]:
# The best fit here gives the starting point for the MCMC sampling
best_fit_idx = np.argmin(chi2)

In [28]:
# Check by plotting best fit model
fig = plt.figure()
ax = fig.add_subplot(111)

# Set labels
ax.set_xlabel(r'$\mathrm{Wavelength\ [\AA]}$', fontsize=15)
ax.set_ylabel(r'$\mathrm{L_{\lambda}\ (divided\ by\ continuum)}$', fontsize=15)

# Plot stack and errors
ax.plot(resampling_grid, stack['flam'], '.-', color='mediumblue', linewidth=1.5, \
        markeredgecolor='mediumblue', markersize=1.0, zorder=5)
ax.fill_between(resampling_grid, stack['flam'] - stack['flam_err'], stack['flam'] + stack['flam_err'], \
                color='gray', alpha=0.5, zorder=5)

# Now plot best fitting model
ax.plot(resampling_grid, models_divcont[best_fit_idx], color='tab:red', linewidth=1.5, zorder=5)

# Horizontal line at 1.0
ax.axhline(y=1.0, ls='--', color='k')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
# ------------------------ Starting the MCMC part ------------------------ #
def get_model(model_params, ):
    
    return model_flam